## List all files in the wiki folder

We can create a list with the names of all files in the wiki folder using the os.listdir() function

In [2]:
import os

file_names = os.listdir("wiki")
len(file_names)


999

## Read the first file

Let's read the first file and print its contents. We need to join the name of the file with the wiki folder. We can do this using the os.path.join() function

In [3]:
with open(os.path.join("wiki", file_names[0])) as f:
    print(f.read())


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Bay of Concepción - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Bay_of_Concepción","wgTitle":"Bay of Concepción","wgCurRevisionId":647460156,"wgRevisionId":647460156,"wgArticleId":16044270,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Coordinates on Wikidata","All stub articles","Landforms of Bío Bío Region","Bays of Chile","Bío Bío Region geography stubs"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNa


## Adding the MapReduce function to this project

We start by adding the MapReduce function so that we can use throughout the project.


In [7]:


import math
import functools
from multiprocessing import Pool

def make_chunks(data, num_chunks):
    chunk_size = math.ceil(len(data) / num_chunks)
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

def map_reduce(data, num_processes, mapper, reducer):
    chunks = make_chunks(data, num_processes)
    with Pool(num_processes) as pool:
        chunk_results = pool.map(mapper, chunks)
 
    return functools.reduce(reducer, chunk_results)




## Counting the total number of lines on all files

It was not required but can use MapReduce to count the total number of lines on all files in the wiki folder! If you did in some other way, that is fine as well.


In [8]:
def map_line_count(file_names):
    total = 0
    for fn in file_names:
        with open(os.path.join("wiki", fn)) as f:
            total += len(f.readlines())
    return total
    
def reduce_line_count(count1, count2):
    return count1 + count2

target = "data"
map_reduce(file_names, 8, map_line_count, reduce_line_count)

499797


## Grep string function

We defined a mapreduce_grep_string() function that takes two arguments as input:

    A path to a folder. In the case of this guided project we will only use it on the wiki folder but having this argument makes the function easier to reuse.

    The string that we want to find.

The mapper function receives a chunk of filenames and calculates all occurrences of the target string on them. If a file contains no occurrences, we chose to not include an entry for that file in the result dictionary.

The reducer function uses the dict.update() method to merge the result dictionaries.

Note that the target variable will be defined outside and will be the string we are looking for.


In [9]:
# The target variable is defined outside and contains the string 
def map_grep(file_names):
    results = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line for line in f.readlines()]
        for line_index, line in enumerate(lines):
            if target in line:
                if fn not in results:
                    results[fn] = []
                results[fn].append(line_index)
    return results

def reduce_grep(lines1, lines2):
    lines1.update(lines2)
    return lines1

def mapreduce_grep(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes,  map_grep, reduce_grep)

## Finding the occurences of "data"

In [11]:
target = "data"
data_occurrences = mapreduce_grep("wiki", 8)
print(data_occurrences)

{'wiki/Bay_of_ConcepciC3B3n.html': [6, 45, 58, 60, 62, 105, 188, 205], 'wiki/Bye_My_Boy.html': [276, 359, 376], 'wiki/Valentin_Yanin.html': [101, 144, 227, 244], 'wiki/Kings_XI_Punjab_in_2014.html': [221, 229, 237, 245, 253, 269, 277, 293, 301, 317, 325, 341, 374, 376, 381, 383, 388, 390, 395, 397, 402, 564, 647, 664], 'wiki/William_Harvey_Lillard.html': [45, 65, 81, 129, 212, 229], 'wiki/Radial_Road_3.html': [52, 103, 301, 505, 588, 605], 'wiki/George_Weldrick.html': [194, 277, 294], 'wiki/Zgornji_Otok.html': [6, 53, 55, 65, 69, 211, 260, 262, 311, 394, 411], 'wiki/Blue_Heelers_(season_8).html': [49, 79, 82, 105, 107, 125, 127, 133, 135, 141, 143, 660, 695, 730, 739, 886, 969, 986], 'wiki/Taggen_Nunatak.html': [6, 44, 46, 48, 93, 176, 193], 'wiki/Henri_BraqueniC3A9.html': [43, 46, 92, 175, 192], 'wiki/Vrila.html': [6, 57, 59, 69, 73, 99, 100, 102, 151, 234, 251], 'wiki/William_Henry_Porter.html': [48, 88, 171, 188], 'wiki/Clive_Brown_(footballer).html': [146, 229, 246], 'wiki/Blick_na

## Allow for case insensitive matches

We can allow case insensitive matches by converting both the target and the file contents to lowercase before we match

In [13]:


def map_grep_insensitive(file_names):
    results = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line.lower() for line in f.readlines()]
        for line_index, line in enumerate(lines):
            if target.lower() in line:
                if fn not in results:
                    results[fn] = []
                results[fn].append(line_index)
    return results

def mapreduce_grep_insensitive(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes,  map_grep_insensitive, reduce_grep)

target = "data"
new_data_occurrences = mapreduce_grep_insensitive("wiki", 8)




## Checking that we find more matches

We already stored the results into variables data_occurrences and new_data_occurrences. To check that we find more matches with the second version of the algorithm, we can loop over the file names and print the length difference between the results.


In [14]:


for fn in new_data_occurrences:
    if fn not in data_occurrences:
        print("Found {} new matches on file {}".format(len(new_data_occurrences[fn]), fn))
    elif len(new_data_occurrences[fn]) > len(data_occurrences[fn]):
        print("Found {} new matches on file {}".format(len(new_data_occurrences[fn]) - len(data_occurrences[fn]), fn))



Found 1 new matches on file wiki/Table_Point_Formation.html
Found 1 new matches on file wiki/Ingrid_GuimarC3A3es.html
Found 2 new matches on file wiki/Jules_Verne_ATV.html
Found 1 new matches on file wiki/Pictogram.html
Found 2 new matches on file wiki/Claire_Danes.html
Found 1 new matches on file wiki/PTPRS.html
Found 1 new matches on file wiki/A_Beautiful_Valley.html
Found 1 new matches on file wiki/Mudramothiram.html
Found 2 new matches on file wiki/Gordon_Bau.html
Found 1 new matches on file wiki/Embraer_Unidade_GaviC3A3o_Peixoto_Airport.html
Found 3 new matches on file wiki/Code_page_1023.html
Found 1 new matches on file wiki/Cryptographic_primitive.html
Found 1 new matches on file wiki/Alex_Kurtzman.html
Found 1 new matches on file wiki/Filip_Pyrochta.html
Found 1 new matches on file wiki/Morgana_King.html
Found 1 new matches on file wiki/Don_Parsons_(ice_hockey).html
Found 1 new matches on file wiki/Bias.html
Found 2 new matches on file wiki/Tomohiko_ItC58D_(director).html
Found


## Finding match indexes on lines

We need to solve a subproblem before we implement this one: Given a string and a target, find all occurrences of the target within that string.


In [15]:
def find_match_indexes(line, target):
    results = []
    i = line.find(target, 0)
    while i != -1:
        results.append(i)
        i = line.find(target, i + 1)
    return results

# Test implementation
s = "Data science is related to data mining, machine learning and big data.".lower()
print(find_match_indexes(s, "data"))

[0, 27, 65]



## Finding all match locations

We can use any of the above functions to find all match locations. We will use the third one.

After finding all indexes in one line, we need to create pairs by adding the line index.


In [16]:
def map_grep_match_indexes(file_names):
    results = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line.lower() for line in f.readlines()]
        for line_index, line in enumerate(lines):
            match_indexes = find_match_indexes(line, target.lower())
            if fn not in results:
                results[fn] = []
            results[fn] += [(line_index, match_index) for match_index in match_indexes]
    return results

def mapreduce_grep_match_indexes(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes,  map_grep_match_indexes, reduce_grep)

target = "science"
occurrences = mapreduce_grep_match_indexes("wiki", 8)


## Displaying the results

Let's display the results. We will create a CSV file listing all occurrences. We will also show the text around each occurrence.


In [18]:
import csv

# How many character to show before and after the match
context_delta = 30

with open("results.csv", "w") as f:
    writer = csv.writer(f)
    rows = [["File", "Line", "Index", "Context"]]
    for fn in occurrences:
        with open(fn) as f:
            lines = [line.strip() for line in f.readlines()]
        for line, index in occurrences[fn]:
            start = max(index - context_delta, 0)
            end   = index + len(target) + context_delta
            rows.append([fn, line, index, lines[line][start:end]])
    writer.writerows(rows)

In [19]:
import pandas
df = pandas.read_csv("results.csv")
df.head(10)

,File,Line,Index,Context
0,wiki/Valentin_Yanin.html,6,840,"embers of the USSR Academy of Sciences"",""Full ..."
1,wiki/Valentin_Yanin.html,6,890,"ers of the Russian Academy of Sciences"",""Demid..."
2,wiki/Valentin_Yanin.html,66,90,"href=""/wiki/Soviet_Academy_of_Sciences"" class=..."
3,wiki/Valentin_Yanin.html,66,145,"ect"" title=""Soviet Academy of Sciences"">Soviet..."
4,wiki/Valentin_Yanin.html,66,173,"f Sciences"">Soviet Academy of Sciences</a>; he..."
5,wiki/Valentin_Yanin.html,144,1440,"rs_of_the_USSR_Academy_of_Sciences"" title=""Cat..."
6,wiki/Valentin_Yanin.html,144,1502,"rs of the USSR Academy of Sciences"">Full Membe..."
7,wiki/Valentin_Yanin.html,144,1548,rs of the USSR Academy of Sciences</a></li><li...
8,wiki/Valentin_Yanin.html,144,1632,"of_the_Russian_Academy_of_Sciences"" title=""Cat..."
9,wiki/Valentin_Yanin.html,144,1697,"of the Russian Academy of Sciences"">Full Membe..."
